In [138]:
from __future__ import print_function

import argparse
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import QMNIST
from tqdm import tqdm_notebook as tqdm

In [139]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(1,32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Conv2d(32,32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=0.20)
            )

        self.cnn2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            nn.Dropout2d(0.25)
        )

        self.cnn3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Dropout2d(p=0.25),
        )

        self.fc = nn.Sequential(
            #全結合層
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.Dropout(),
            nn.Linear(128, 10)
        )
 

    def forward(self, x):  

        out = self.cnn1(x)
        out = self.cnn2(out)
        out = self.cnn3(out)
        out = out.view(out.size()[0], -1)
        out = self.fc(out)
        out = F.softmax(out)
        return out


In [140]:
def train_model(net, dataloders_dict, criterion, optimizer, num_epochs):
    
    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []

    #初期設定
    #GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス", device)
    #モデルをGPUへ
    net.to(device)
    #ネットワークがある程度固定であれば高速化させる
    torch.backends.cudnn.benchmark = True
    #epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-----------------------------------')

        #epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() #モデルを訓練モードに
            else:
                net.eval() #モデルを検証モードに
            epoch_loss = 0.0 #epochの損失0
            epoch_corrects = 0 #epochの正解数
            #データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloders_dict[phase]):
                #optimizerを初期化
                optimizer.zero_grad()
                #順伝搬(forward)計算
                with torch.set_grad_enabled(phase == 'train'):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)#損失を計算
                    _, preds = torch.max(outputs, 1) #ラベルを予測
                    #訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    #イテレーション結果の計算
                    #lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    #正解の合計数を更新
                    epoch_corrects += torch.sum(preds == labels.data)
            #epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloders_dict[phase].dataset)
            print('{} Loss:{:.4f} Acc: {:.4f}'.format(phase, epoch_loss,epoch_acc))
            if phase == 'train':
                train_acc_list.append(epoch_acc)
                train_loss_list.append(epoch_loss)
            else:
                val_acc_list.append(epoch_acc)
                val_loss_list.append(epoch_loss)
    return val_loss_list,train_loss_list, val_acc_list, train_acc_list

In [141]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
from torchvision import transforms
from PIL import Image

import os.path as osp
import glob


#入力画像の処理を行う
#訓練時と推論時で処理が異なる

class ImageTransform():

    def __init__(self, resize):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize(resize), #リサイズ
                # transforms.RandomRotation(degrees=20), #ランダムに回転
                transforms.ToTensor(), #テンソルに変換
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize), #リサイズ
                transforms.ToTensor(), #テンソルに変換
            ])
        }

    def __call__(self, img, phase='train'):
        """
        pahes:'train' or 'val'
        """
        return self.data_transform[phase](img)

class testDataset(data.Dataset):
    '''
    file_list : リスト
        画像パス
    transform: object
        前処理クラスのインスタンス
    phase : 学習化テストか設定する
    '''
    def __init__(self,transform=None, phase='val'):

        self.transform = transforms.Compose([
                        transforms.Resize(32),
                        transforms.CenterCrop(28),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, ), (0.5, ))
                         ])
        self.phase = phase#train or val の指定

        rootpath = "./data/tranings/"
        target_path = osp.join(rootpath+'**/*.png')
    
        path_list = []

        for path in glob.glob(target_path):
            path_list.append(path)

        self.file_list = path_list

    def __len__(self):
        #画像の枚数を返す
        return len(self.file_list)
    
    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルの取得
        '''
        #index番目の画像をロード
        img_path = self.file_list[index]
        label = img_path[16]

        img = Image.open(img_path).convert('RGBA')
        r, g, b, a = img.split()
        img = np.asarray(a)
        img = Image.fromarray(img)
        
        img_transformed = self.transform(
            img
        )

        label = torch.tensor(int(label), dtype=torch.int64)
        
        return img_transformed, label

In [142]:
import random
seed = 42
random.seed(seed)  
np.random.seed(seed)  
# PyTorch のRNGを初期化  
torch.manual_seed(seed)

import tensorflow as tf
import torch.utils.data as data
from torch.utils.data import DataLoader

rotation_range=10
zoom_range = 0.10
width_shift_range=0.1 
height_shift_range=0.1
train_transform = transforms.Compose([
#      transforms.Resize(36)
#      transforms.RandomCrop(28),
     transforms.RandomAffine(degrees=rotation_range, translate=(width_shift_range, height_shift_range), 
    scale=(1 - zoom_range, 1 + zoom_range)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))
     ])
# test_transform = transforms.Compose([
#     transforms.Resize(32)
#      transforms.CenterCrop(28),
#      transforms.ToTensor(),
#      transforms.Normalize((0.5, ), (0.5, ))
#      ])


trainset = QMNIST(root='./',
                 train=True,
                 download=True,
                 transform=train_transform)


trainloader = DataLoader(trainset,
                         batch_size=1024,
                         shuffle=True,
                         num_workers=2)
testloader =  torch.utils.data.DataLoader(testDataset(), batch_size = 32, shuffle=False)

dataloders_dict = {"train": trainloader, "val": testloader}

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

val_loss, train_loss, val_acc, train_acc = train_model(model, dataloders_dict, criterion, optimizer, num_epochs=10)
torch.save(model.state_dict(), "mnist_cnn.pth")

使用デバイス cuda:0
Epoch 1/10
-----------------------------------


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



train Loss:1.8280 Acc: 0.6883



val Loss:1.8717 Acc: 0.7460
Epoch 2/10
-----------------------------------


...


val Loss:1.5597 Acc: 0.9120
Epoch 9/10
-----------------------------------



train Loss:1.4857 Acc: 0.9788



val Loss:1.5670 Acc: 0.9020
Epoch 10/10
-----------------------------------



train Loss:1.4842 Acc: 0.9801



val Loss:1.5640 Acc: 0.9000


In [137]:
import os
import cv2
import sys
import numpy as np
from PIL import Image

In [80]:
img = Image.open("data/tests/25784885.png").resize((28, 28)).convert('RGBA')
img

In [81]:
r, g, b, a = img.split()
img = np.asarray(a)

In [82]:
image = img

In [83]:
inputs = torch.tensor(image, dtype=torch.float32)

In [84]:
inputs.shape

torch.Size([28, 28])

In [94]:
model = Net()
model.load_state_dict(torch.load("./model/mnist_cnn.pth"))
model.to('cuda')

Net(
  (cnn1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout2d(p=0.2, inplace=False)
  )
  (cnn2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout2d(p=0.25, inplace=False)
  )
  (cnn3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Dropout2d(p=0.25, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=

In [86]:
inputs = test_transform(image)
inputs = inputs[None, ...]
inputs = inputs.to('cuda')

In [87]:
model.eval()
output = model(inputs)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [88]:
_, predicted = torch.max(output.data, 1)

In [89]:
predicted.item()

7